In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
train =pd.read_csv(r'C:\Users\omarf\OneDrive\Documents\train.csv')
train.head()

,id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,0,42,technician,married,secondary,no,7,no,no,cellular,25,aug,117,3,-1,0,unknown,0
1,1,38,blue-collar,married,secondary,no,514,no,no,unknown,18,jun,185,1,-1,0,unknown,0
2,2,36,blue-collar,married,secondary,no,602,yes,no,unknown,14,may,111,2,-1,0,unknown,0
3,3,27,student,single,secondary,no,34,yes,no,unknown,28,may,10,2,-1,0,unknown,0
4,4,26,technician,married,secondary,no,889,yes,no,cellular,3,feb,902,1,-1,0,unknown,1


In [5]:
test = pd.read_csv(r'C:\Users\omarf\OneDrive\Documents\test.csv')
test.head()

,id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
0,750000,32,blue-collar,married,secondary,no,1397,yes,no,unknown,21,may,224,1,-1,0,unknown
1,750001,44,management,married,tertiary,no,23,yes,no,cellular,3,apr,586,2,-1,0,unknown
2,750002,36,self-employed,married,primary,no,46,yes,yes,cellular,13,may,111,2,-1,0,unknown
3,750003,58,blue-collar,married,secondary,no,-1380,yes,yes,unknown,29,may,125,1,-1,0,unknown
4,750004,28,technician,single,secondary,no,1950,yes,no,cellular,22,jul,181,1,-1,0,unknown


In [8]:
submission = pd.read_csv(r'C:\Users\omarf\OneDrive\Documents\sample_submission.csv')
submission.head()

,id,y
0,750000,0.5
1,750001,0.5
2,750002,0.5
3,750003,0.5
4,750004,0.5


In [9]:
train.info()
train.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750000 entries, 0 to 749999
Data columns (total 18 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   id         750000 non-null  int64 
 1   age        750000 non-null  int64 
 2   job        750000 non-null  object
 3   marital    750000 non-null  object
 4   education  750000 non-null  object
 5   default    750000 non-null  object
 6   balance    750000 non-null  int64 
 7   housing    750000 non-null  object
 8   loan       750000 non-null  object
 9   contact    750000 non-null  object
 10  day        750000 non-null  int64 
 11  month      750000 non-null  object
 12  duration   750000 non-null  int64 
 13  campaign   750000 non-null  int64 
 14  pdays      750000 non-null  int64 
 15  previous   750000 non-null  int64 
 16  poutcome   750000 non-null  object
 17  y          750000 non-null  int64 
dtypes: int64(9), object(9)
memory usage: 103.0+ MB


id           0
age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64

In [ ]:
# Create a copy of train for feature engineering
train_fe = train.copy()

# Drop the id column (not useful as a feature)
if 'id' in train_fe.columns:
    train_fe.drop(columns=['id'], inplace=True)

# Convert all string columns to lowercase for consistency
cat_cols = train_fe.select_dtypes(include=['object']).columns
for col in cat_cols:
    train_fe[col] = train_fe[col].str.lower()

# Map month names to numbers
month_mapping = {'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6,
                 'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12}
train_fe['month_num'] = train_fe['month'].map(month_mapping)
train_fe.drop(columns=['month'], inplace=True)

# Convert binary categorical features to numerical values
for col in ['default', 'housing', 'loan']:
    train_fe[col] = train_fe[col].map({'yes': 1, 'no': 0})

# For 'poutcome', treat 'unknown' as a separate category; we'll one-hot encode it later.
# No special transform here apart from ensuring consistent lower-case

# One-hot encode remaining categorical features
train_fe = pd.get_dummies(train_fe, columns=['job', 'marital', 'education', 'contact', 'poutcome'], drop_first=True)

# Create interaction feature between age and balance
train_fe['age_balance_interaction'] = train_fe['age'] * train_fe['balance']

# Create a ratio feature between balance and duration (avoid division by zero)
train_fe['balance_duration_ratio'] = train_fe.apply(lambda row: row['balance'] / row['duration'] if row['duration'] != 0 else 0, axis=1)

# Display the first few rows of the engineered dataset
print(train_fe.head())

   age  default  balance  housing  loan  day  duration  campaign  pdays  \
0   42        0        7        0     0   25       117         3     -1   
1   38        0      514        0     0   18       185         1     -1   
2   36        0      602        1     0   14       111         2     -1   
3   27        0       34        1     0   28        10         2     -1   
4   26        0      889        1     0    3       902         1     -1   

   previous  ...  education_secondary  education_tertiary  education_unknown  \
0         0  ...                 True               False              False   
1         0  ...                 True               False              False   
2         0  ...                 True               False              False   
3         0  ...                 True               False              False   
4         0  ...                 True               False              False   

   contact_telephone  contact_unknown  poutcome_other  poutcome_succ

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

# Separate features and target
X = train_fe.drop(columns=['y'])
y = train_fe['y']

# Train-test split (80:20)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Fit logistic regression
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)

# Predict probabilities for validation set
y_pred_proba = lr.predict_proba(X_val)[:, 1]

# Evaluate ROC AUC
roc_auc = roc_auc_score(y_val, y_pred_proba)
print(f"Validation ROC AUC: {roc_auc:.4f}")

Validation ROC AUC: 0.8992


C:\Users\omarf\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [13]:
# Apply the same feature engineering steps to the test set
test_fe = test.copy()

# Drop the id column after saving it for submission
test_ids = test_fe['id']

# Convert all string columns to lowercase for consistency
for col in cat_cols:
    if col in test_fe.columns:
        test_fe[col] = test_fe[col].str.lower()

# Map month names to numbers
test_fe['month_num'] = test_fe['month'].map(month_mapping)
test_fe.drop(columns=['month'], inplace=True)

# Convert binary categorical features to numerical values
for col in ['default', 'housing', 'loan']:
    test_fe[col] = test_fe[col].map({'yes': 1, 'no': 0})

# One-hot encode remaining categorical features
test_fe = pd.get_dummies(test_fe, columns=['job', 'marital', 'education', 'contact', 'poutcome'], drop_first=True)

# Create interaction feature between age and balance
test_fe['age_balance_interaction'] = test_fe['age'] * test_fe['balance']

# Create a ratio feature between balance and duration (avoid division by zero)
test_fe['balance_duration_ratio'] = test_fe.apply(lambda row: row['balance'] / row['duration'] if row['duration'] != 0 else 0, axis=1)

# Align test features to train features (fill missing columns with 0)
missing_cols = set(X.columns) - set(test_fe.columns)
for col in missing_cols:
    test_fe[col] = 0
test_fe = test_fe[X.columns]

# Predict probabilities using the trained logistic regression model
test_pred_proba = lr.predict_proba(test_fe)[:, 1]

# Create submission file
submission = pd.DataFrame({'id': test_ids, 'y': test_pred_proba})
submission.to_csv('submission.csv', index=False)

In [14]:
import IPython.display as ipd

ipd.display(ipd.FileLink('submission.csv'))

c:\Users\omarf\OneDrive\Documents\submission.csv